# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import json
import time
import asyncio

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
#ODEL_FILE = MODEL_NAME + '.tar.gz'
#OWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Object tracking function

In [7]:
def set_baseline(x1, y1, x2, y2):
    return y2 - y1, x1 - x2, x2 * y1 - x1 * y2

In [8]:
def get_distance_from_line(x1, y1, a, b, c):
    return (a*x1 + b*y1 + c) / (((a ** 2) + (b ** 2)) ** 0.5)

# Baseline setting function

In [9]:
baseline_x = []
baseline_y = []
mouse_count = 2

def get_mouse_click_location(event, x, y, flags, param):
    global mouse_count, a, b, c
    if (mouse_count > 0 and event == cv2.EVENT_LBUTTONDOWN):
        baseline_x.append(x)
        baseline_y.append(y)
        mouse_count = mouse_count - 1
        if (mouse_count == 0):
            a, b, c = set_baseline(baseline_x[0] * 0.001, baseline_y[0] * 0.001, baseline_x[1] * 0.001, baseline_y[1] * 0.001)

# Waring signal function

In [10]:
standard_distance = 0.03

def send_warning_signal(input_distance, input_class):
    if standard_distance > input_distance:
        if input_class == 'person':
            return 0
        else:
            return 1
    return -1

# Asyncio Setting

In [34]:
reader, writer = await asyncio.open_connection( '192.168.219.107', 6974)

# STORED VIDEO ONLY

In [35]:
inputVideo = '.\\video_samples\\s1_a.mp4'
cap=cv2.VideoCapture(inputVideo)
signal = -1

time_signal = -1
time_value = time.time()

with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        if len(baseline_x) == 2:
            cv2.imshow('stored version', cv2.line(image_np,(baseline_x[0], baseline_y[0]),(baseline_x[1], baseline_y[1]),(0,0,255),5))
            
            for index, value in enumerate(classes[0]):
                ymin = boxes[0][index][0]
                xmin = boxes[0][index][1]
                ymax = boxes[0][index][2]
                xmax = boxes[0][index][3]
                class_name = (category_index.get(value)).get('name')
                widthvalue = int((xmax - xmin) / 2)  # width 길이
                heightvalue = int((ymax - ymin) / 2)  # height 길이
                if ((class_name == "person" or class_name == "car") and scores[0, index] > 0.80) :
                    get_distance_from_line((xmin + xmax) / 2, (ymin + ymax) / 2, a, b, c)
                    signal = send_warning_signal(get_distance_from_line((xmin + xmax) / 2, (ymin + ymax) / 2, a, b, c), class_name)
                    if signal >= 0 :
                        if (signal >= time_signal):
                            time_signal = signal
                            time_value = time.time()

        if (time.time() - time_value < 1):
            signal = time_signal
        else :
            time_signal = -1
            time_value = 0
            
        Camera_data = {
            'Camera_ID' : 1,
            'Value' : signal
        }
        
        message = json.dumps(Camera_data) + "\n"        
        writer.write(message.encode())
        await writer.drain()
        signal = -1
        
        cv2.imshow('stored version', image_np)
        
        # Mouse Click 좌표를 위한 함수
        cv2.setMouseCallback('stored version', get_mouse_click_location)
        
        if cv2.waitKey(25) & 0xFF==ord('q'): #waitKey( 내부의 값이 작아지면 CPU 의 부담은 커지는데 비해 처리속도는 빨라짐 )
            break

        # 저장된 비디오의 경우 한정
        if(cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT)):
            break
            
            
    writer.close()    
    await writer.wait_closed()    
    cv2.destroyAllWindows()
    cap.release()

# STREAMING ONLY

In [11]:
cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach
with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        for index, value in enumerate(classes[0]):
            ymin = boxes[0][index][0]
            xmin = boxes[0][index][1]
            ymax = boxes[0][index][2]
            xmax = boxes[0][index][3]
            class_name = (category_index.get(value)).get('name')
            widthvalue = int((xmax - xmin) / 2)  # width 길이
            heightvalue = int((ymax - ymin) / 2)  # height 길이
            if ((class_name == "person" or class_name == "car") and scores[0, index] > 0.80) :
                print((xmin + xmax) / 2, (ymin + ymax) / 2)
                print(class_name  + str(index) + "\n")
                
        cv2.imshow('streaming version', image_np)
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
        
    cv2.destroyAllWindows()
    cap.release()

0.5792062282562256 0.6157652735710144
person0

0.5761924386024475 0.6175026893615723
person0

0.5787792801856995 0.6159119606018066
person0

0.5779803395271301 0.6162577867507935
person0

0.5772362947463989 0.6168587803840637
person0

0.5756396055221558 0.6162172555923462
person0

0.5786385536193848 0.614600658416748
person0

0.5755528807640076 0.6174510717391968
person0

0.5741474032402039 0.6160492897033691
person0

0.576866865158081 0.6150716543197632
person0

0.576814591884613 0.6145763993263245
person0

0.5753225088119507 0.6179227828979492
person0

0.5763444900512695 0.6158076524734497
person0

0.5753112435340881 0.6182451844215393
person0

0.5769355893135071 0.6166581511497498
person0

0.5777291655540466 0.6162953972816467
person0

0.5758413672447205 0.6178328394889832
person0

0.5781272649765015 0.6167515516281128
person0

0.577896773815155 0.6172510385513306
person0

0.5758306384086609 0.6179730892181396
person0

0.5752586126327515 0.6180391907691956
person0

0.575812757015228